In [1]:
#### run combined df again if any code changes occur (symbol change) including additions 


In [12]:
#### combines 7 days delivery files
#### reads market cap from screener
#### reads averages

#### this reads non_fno_wl, yymmdd_volume_3x_wl, yymmdd_tview_wl_above_500, yymmdd_tview_wl_below_500, sme_csv
 

#### heading for reports
#### title for reports

In [1]:
# importing necessary modules 

import os
import pandas as pd
import csv
from pathlib import Path

base_folder = Path.cwd()
print(base_folder)

c:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Python code for chatGpt


In [2]:
###  datapath for lenovo desktop

master_file_dir = Path.joinpath(base_folder, "Master File")

master_file_path = Path.joinpath(base_folder, "Master File", "CompanyMaster.csv")
directory = Path.joinpath(Path(base_folder).parents[0], "Data Files")
combined_directory =  Path.joinpath(Path(base_folder).parents[0], "Delivery Price Combined Files")


input_path =  Path.joinpath(base_folder, "Input File")
output_path = Path.joinpath(base_folder, "Output File")

In [3]:
column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']


non_fno_csv = os.path.join(output_path, '0_non_fno.csv')
fno_csv = os.path.join(output_path, 'fno.csv')
sme_csv = os.path.join(output_path, 'sme_csv.csv')

avgs_file = os.path.join(input_path, 'NSE BSE Averages.xlsx')

In [4]:
def find_corresponding_codes(master_file_path, user_input):
    nse_code = ""
    bse_code = ""
    co_name = ""

    user_input = user_input.lower()  # Convert user input to lowercase for comparison

    with open(master_file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[1].lower() == user_input:  # NSE code match (case-insensitive)
                nse_code = row[1]
                bse_code = str(row[2])
                co_name = row[0]
                break
            elif row[2].lower() == user_input:  # BSE code match (case-insensitive)
                nse_code = row[1]
                bse_code = str(row[2])
                co_name = row[0]
                break

    return nse_code, bse_code,co_name

def format_value(value):
    return '{:,.0f}'.format(value)

def format_decimal(x):
    return '{:,.2f}'.format(x)

def averages(scrip_code):
    ema10 = [0]  # Default value for ema10
    ema20 = [0]  # Default value for ema20
    ema50 = [0]  # Default value for ema50
    ema200 = [0]  # Default value for ema200
    
    
        
    if type(scrip_code) == int:
        scrip_code = str(scrip_code)
        averages_df = bse_avg_df[bse_avg_df['Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        averages_df = nse_avg_df[nse_avg_df['Name'] == scrip_code]
    
    if not averages_df.empty:
        ema10 = averages_df['ema10'].values.tolist()
        ema20 = averages_df['ema20'].values.tolist()
        ema50 = averages_df['ema50'].values.tolist()
        ema200 = averages_df['ema200'].values.tolist()
    
    return ema10, ema20, ema50, ema200


def get_mktcap(scrip_code):
    
    if type(scrip_code) == int:
        #scrip_code = str(scrip_code)
        scrip_mktcap_df = mktcap_df[mktcap_df['BSE_Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        scrip_mktcap_df = mktcap_df[mktcap_df['NSE_Code'] == scrip_code]
        
    #scrip_mktcap_df.info()
    
    if not scrip_mktcap_df.empty:
        scrip_mktcap = scrip_mktcap_df['Market Capitalization'].values.tolist()
        
    else:
        scrip_mktcap = [0]
    
    return scrip_mktcap


def get_networth(scrip_code):
    
    if type(scrip_code) == int:
        #scrip_code = str(scrip_code)
        scrip_mktcap_df = mktcap_df[mktcap_df['BSE_Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        scrip_mktcap_df = mktcap_df[mktcap_df['NSE_Code'] == scrip_code]
        
    #scrip_mktcap_df.info()
    
    if not scrip_mktcap_df.empty:
        scrip_networth = scrip_mktcap_df['Net worth'].values.tolist()
        
    else:
        scrip_networth = [0]
    
    return scrip_networth
     
    
    
def replace_nse_code(row):
    if pd.isna(row['NSE_Code']) or row['NSE_Code'] == "":
        return row['BSE_Code']
    else:
        return row['NSE_Code']

# Apply the custom function to the NSE_Code column



def list_process(list_name):
    
    list_name_lower = [item.lower() for item in list_name]
    nse_code_df = combined_df[combined_df['NSE_Code'].str.lower().isin(list_name_lower)]
    
    numeric_list_name = []
    
    for value in list_name:
        if value.isdigit():
            numeric_list_name.append(int(value))
    
    bse_code_df = combined_df[combined_df['BSE_Code'].isin(numeric_list_name)]
    final_code_df = pd.concat([nse_code_df,bse_code_df], ignore_index=True)
    final_code_df['NSE_Code'] = final_code_df.apply(replace_nse_code, axis=1)
    
    final_code_df.sort_values(by=['NSE_Code', 'Date'], ascending=[True, False], inplace=True)
    grouped_df = final_code_df.groupby('NSE_Code')
    
    for name, group in grouped_df:
        
        ema10, ema20, ema50, ema200 = averages(str(name))
        ema_str = "ema10 / ema20 / ema50 / ema200"
    
        print("")
        print(f"Company Name: {group['Company Name'].iloc[0]} (NSE Code: {name}, BSE Code: {group['BSE_Code'].iloc[0]})")
        print(f"{ema_str}     {ema10[0]}  {ema20[0]}  {ema50[0]}  {ema200[0]}")
        print("")

        # Format the columns before printing
        group['NSE_Delivery'] = group['NSE_Delivery'].apply(lambda x: '{:,.0f}'.format(x))
        group['BSE_Delivery'] = group['BSE_Delivery'].apply(lambda x: '{:,.0f}'.format(x))
        group['Total_Delivery'] = group['Total_Delivery'].apply(lambda x: '{:,.0f}'.format(x))

        # Print the header with column names excluding 'NSE_Code' and 'BSE_Code'
        header = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
    
        # Print the table with manual formatting
        print(f"{'Date':<8}{'NSE_Delivery':>15}{'BSE_Delivery':>15}{'Total_Delivery':>20}{'Price':>15}")
        for _, row in group[header].iterrows():
            print(f"{row['Date']:<8}{row['NSE_Delivery']:>15}{row['BSE_Delivery']:>15}{row['Total_Delivery']:>20}{row['Price']:>15.2f}")

        # Add two blank spaces between groups
        print("\n")
        

    
    return final_code_df
    
    
def list_process_new(list_name, header):
    
    print(header)
    
    for company in list_name:
    # Filter the DataFrame for the current company
    
        if company.isdigit():
            company_int = int(company)
            company_data = combined_df[combined_df['BSE_Code'] == company_int]
            #print(company)
        else:
            company = company.upper()
            company_data = combined_df[combined_df['NSE_Code'] == company]
            #print(company)

        # Check if the DataFrame is not empty
        if not company_data.empty:
            # Extract relevant information
            company_name = company_data['Company Name'].values[0]
            nse_code = company_data['NSE_Code'].values[0]
            bse_code = company_data['BSE_Code'].values[0]

            ema10,ema20,ema50,ema200 = averages(nse_code)
            ema_str = "ema10 / ema20 / ema50 / ema200"
            
            mktcap_value = get_mktcap(nse_code)
            networth_value = get_networth(nse_code)
            
            print(f"Company Name: {company_name} (NSE Code: {nse_code}, BSE Code: {bse_code}, Mktcap:{mktcap_value[0]:,.0f}, Networth:{networth_value[0]:,.0f})")

            print("")
        
            print(f"{ema_str}     {ema10[0]}  {ema20[0]}  {ema50[0]}  {ema200[0]}")
            print("")

            # Format and print the specified columns with additional spaces
            formatted_columns = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
            formatted_data = company_data[formatted_columns].copy()
            formatted_data['NSE_Delivery'] = formatted_data['NSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['BSE_Delivery'] = formatted_data['BSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Total_Delivery'] = formatted_data['Total_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{x:.2f}")

            # Adjust the spacing between columns
            formatted_data_str = formatted_data.to_string(index=False, 
                                                      col_space={'Date':9,
                                                                 'NSE_Delivery':15,
                                                                 'BSE_Delivery':15,
                                                                 'Total_Delivery': 15, 
                                                                 'Price': 10
                                                                 }
                                                        )

            print(formatted_data_str)
            # Add a line break between companies
            print("\n" + "=" * 50 + "\n")
            print("")
        else:
            print(f"No data found for {company}\n")

    
def add_nse_prefix(df, column_name):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)

    return df


In [5]:
def print_formatted_rows(stock_list, title):
    # Print the header
    header = "Company Name".ljust(35) + "StockCode".ljust(10) + "Close".rjust(10) + "PrevClose".rjust(10) + \
             "MktCap".rjust(10) + "EMA10".rjust(10) + "EMA20".rjust(10) + "EMA50".rjust(10) + "EMA200".rjust(10)
    print(title)
    print("")
    print("")
    print(header)
    print("")
    print("")

    line_counter = 0
    
    for stock in stock_list:
        if stock.isdigit():
            stock_code = int(stock)
            df_row = combined_df[combined_df['BSE_Code'] == stock_code]
        else:
            stock_code = stock.upper()
            df_row = combined_df[combined_df['NSE_Code'] == stock_code]

        if not df_row.empty:
            company_name = df_row['Company Name'].values[0]
            stock_code = df_row['NSE_Code'].values[0]
            price = df_row['Price'].values[0]
            prevclose = df_row['PrevClose'].values[0]

            ema10, ema20, ema50, ema200 = averages(stock_code)
            ema10 = ema10[0]
            ema20 = ema20[0]
            ema50 = ema50[0]
            ema200 = ema200[0]

            mktcap_value = get_mktcap(stock_code)
            mktcap = mktcap_value[0]

            # Format the values with ljust and rjust
            formatted_row = company_name.ljust(35) + str(stock_code).ljust(10) + \
                             f"{price:.2f}".rjust(10) + f"{prevclose:.2f}".rjust(10) + \
                             f"{mktcap:,.0f}".rjust(10) + f"{ema10:.2f}".rjust(10) + \
                             f"{ema20:.2f}".rjust(10) + f"{ema50:.2f}".rjust(10) + f"{ema200:.2f}".rjust(10)

            print(formatted_row)
            
            line_counter += 1

            # Insert a blank line after every 5 lines
            if line_counter % 5 == 0:
                print()

In [6]:
#### reading nse and bse averages
file_path = avgs_file

# Replace 'Sheet1' with the name of the sheet you want to read
#sheet_name = 'Sheet1'
nse_averages = "NSE Averages"
bse_averages = "BSE Averages"

# Read the Excel file into a DataFrame
#df = pd.read_excel(file_path, sheet_name=sheet_name)

#nse_avg_df = pd.read_excel(file_path, sheet_name = nse_averages, dtype=column_width, engine='openpyxl')
nse_avg_df = pd.read_excel(file_path, sheet_name=nse_averages)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace('_T$',"",regex=True)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace('_Z$',"",regex=True)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace(' SM$',"",regex=True)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace(' ST$',"",regex=True)


nse_avg_df['Name'] = nse_avg_df['Name'].str.strip()


bse_avg_df = pd.read_excel(file_path, sheet_name = bse_averages)
bse_avg_df['Code'] = bse_avg_df['Code'].astype(str)


####################reading market cap file

mktcap_file = os.path.join(master_file_dir, 'MarketCap from Screener.csv')
mktcap_df = pd.read_csv(mktcap_file)
mktcap_df['BSE Code'] = mktcap_df['BSE Code'].fillna(0)
mktcap_df['BSE Code'] = mktcap_df['BSE Code'].astype(int)
mktcap_df['NSE Code'] = mktcap_df['NSE Code'].fillna("")

mktcap_df.rename(columns= { 'BSE Code' : 'BSE_Code', 
                          'NSE Code': 'NSE_Code'}, inplace =True)

mktcap_df.loc[mktcap_df['NSE_Code'] == "", "NSE_Code"] = mktcap_df['BSE_Code']


In [7]:

###### combine 7 days price files

no_days = 7

file_names = sorted([file for file in os.listdir(combined_directory) if file.lower().endswith(("CSV","csv"))
                        and file.startswith("combined")], reverse=True)[:no_days]

print(file_names)

count = 1

for file_name in file_names:
    file_path = os.path.join(combined_directory,file_name)
    csv_file_df = pd.read_csv(file_path)
    
    if count == 1:
        combined_df = csv_file_df
    else:
        combined_df = pd.concat([combined_df, csv_file_df], ignore_index=True)
    
    count += 1
    #print(count)

combined_df['BSE_Code'] = combined_df['BSE_Code'].fillna(0)
combined_df['BSE_Code'] = combined_df['BSE_Code'].astype(int)
combined_df['NSE_Code'] = combined_df['NSE_Code'].fillna("")
combined_df['Company Name'] = combined_df['Company Name'].fillna("")
combined_df['NSE_Code'] = combined_df.apply(replace_nse_code, axis=1)

# Assuming your DataFrame is named combined_df
report_date = combined_df['Date'].max()
report_date = str(report_date)

print(report_date)

['combined_231020.csv', 'combined_231010.csv', 'combined_230810.csv', 'combined_230420.csv', 'combined_190124.csv']
231020


In [8]:
### reading csv of top 50 by value and volume in delivery

non_fno_df_in = pd.read_csv(non_fno_csv)
fno_df = pd.read_csv(fno_csv)
sme_df_in = pd.read_csv(sme_csv)

"""It fills the missing (NaN) values in the 'NSE_Code' column of the DataFrame non_fno_df with values 
from the 'BSE_Code' column converted to strings.

The .astype(str) part is used to convert the values in the 'BSE_Code' column to strings. 
This is necessary because the 'NSE_Code' column likely contains strings (object dtype), and 
you need to ensure that the replacement values are of the same data type.

The fillna method is used to perform the filling operation. By setting the inplace parameter to True, 
the operation is performed directly on the DataFrame without the need to assign the result back to the DataFrame."""


def fill_missing_nse_code(data_frame):
    # Convert non-finite 'BSE_Code' values to integers while replacing NaN with a placeholder
    placeholder_int = -1  # Placeholder integer value for NaN
    data_frame['BSE_Code'] = data_frame['BSE_Code'].fillna(placeholder_int).astype(int)
    
    # Convert 'BSE_Code' values to strings and fill missing 'NSE_Code' values
    data_frame['NSE_Code'].fillna(data_frame['BSE_Code'].astype(str), inplace=True)

    return data_frame


non_fno_df = fill_missing_nse_code(non_fno_df_in)
sme_df = fill_missing_nse_code(sme_df_in)

#non_fno_df['NSE_Code'].fillna(non_fno_df['BSE_Code'].astype(int).astype(str), inplace=True)
#sme_df['NSE_Code'].fillna(sme_df['BSE_Code'].astype(int).astype(str), inplace=True)

# Get the unique 'NSE_Code' values as a list
unique_non_fno = non_fno_df['NSE_Code'].unique().tolist()
sme_list = sme_df['NSE_Code'].unique().tolist()

focus_list = [ '531112', '538734', '543953', '543971', 'adanipower', 'amber', 'anantraj', 'angelone', 'anup', 'apollo', 
           'arvind', 'arvindfasn', 'bajajhind', 'bclind', 'bluejet', 'bluestarco', 'brigade', 'bse', 'cdsl', 'chembond', 
           'cignititec', 'creditacc', 'cupid', 'daawat', 'dbrealty', 'dharmaj', 'diamondyd', 'elecon', 'electcast', 
           'emil', 'focus', 'gabriel', 'gensol', 'genuspower', 'hblpower', 'hirect', 'hitech', 'hpl', 'idea', 'hcc',
           'ifglexpor', 'iiflsec', 'ikio', 'indotech', 'inoxwind', 'jindalsaw', 'jkcement', 'jklakshmi', '530129',
           'jktyre', 'jpassociat', 'jppower', 'jwl', 'kapston', 'kesoramind', 'kpittech', 'l&tfh', 'lalpathlab', 'tv18brdcst',
           'lodha', 'marine', 'mblinfra', 'mcx', 'newgen', 'paytm', 'peninland', 'pfs', 'repcohome','ndrauto','magson',
           'phoenixltd', 'pplpharma', 'premexpln', 'prestige', 'pyramid', 'radico', 'rategain', 'recltd', '522101',
           'rkforge', 'rpglife', 'sadhnaniq', 'sanghvimov', 'satin', 'sdbl', 'senco', 'sepc', 'shriramfin','shrirampps', 
            'skygold', 'solarinds', 'sonatsoftw', 'spandana', 'suzlon', 'tatamotors', 'thomascook', 'shripiston',
           'titagarh', 'tril', 'unionbank', 'vijaya', 'walchannag', 'welspunind', 'zaggle', 'zeemedia', 'zomato',
           'hariompipe', 'semac', 'tci', '540358', 'icil', '20microns', 'prsmjohnsn', 'poonawalla', 'cello', 'hcc', 
          'bikaji', 'neulandlab', 'jppower', 'trent']

In [12]:
dely_3x = os.path.join(output_path, f'_{report_date}_0_volume_3x_wl.csv')
dely_3x_df = pd.read_csv(dely_3x)
dely_3x_df.head()

,Symbol
0,NSE:PLAZACABLE
1,NSE:541701
2,NSE:514010
3,NSE:512329
4,NSE:GKWLIMITED


In [13]:

#dely_3x_list = dely_3x_df.tolist()
# Extract the 'ScripCode' column from the DataFrame
dely_3x_list = dely_3x_df['Symbol'].tolist()

cleaned_list = [stock.replace('NSE:', '') for stock in dely_3x_list]
dely_3x_list = cleaned_list

# Extract the first 50 records of the 'ScripCode' column from the DataFrame
#dely_3x_list = dely_3x_df['ScripCode'].head(50).tolist()

#dely_3x_list

In [21]:
### above 500 cr

# percent_new_high = os.path.join(output_path, f'_{report_date}_0_tview_wl_above_500.csv')
percent_new_high =r"C:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Python code for chatGpt\Output File\_231221_0_tview_wl_above_500.csv"
percent_new_high_df = pd.read_csv(percent_new_high)

percent_new_high_df.head()

,Ticker
0,NSE:VERTOZ
1,NSE:ARMANFIN
2,NSE:TARSONS
3,NSE:LOKESHMACH
4,NSE:PHANTOMFX


In [27]:
# Replace "NSE:" with an empty string in the 'Ticker' column
percent_new_high_df['Ticker'] = percent_new_high_df['Ticker'].str.replace('NSE:', '')

# Assuming percent_new_high_df is your DataFrame

# Replace "NSE:" with an empty string in the 'Ticker' column
percent_new_high_df['Ticker'] = percent_new_high_df['Ticker'].str.replace('NSE:', '')

# Print the updated DataFrame
#print(percent_new_high_df)

# Convert the 'Ticker' column to a list
ticker_list = percent_new_high_df['Ticker'].tolist()

#print(ticker_list)


#### below 500 cr

# percent_below = os.path.join(output_path, f'_{report_date}_0_tview_wl_below_500.csv')

# percent_below_df = pd.read_csv(percent_below)
percent_below = r"C:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Python code for chatGpt\Output File\_231221_0_tview_wl_below_500.csv"
percent_below_df = pd.read_csv(percent_below)


In [28]:
percent_below_df.head()

,Ticker
0,NSE:SIKKO
1,NSE:STARTECK
2,NSE:HINDCON
3,NSE:CORALFINAC
4,NSE:VISESHINFO


In [ ]:
percent_below_df['Ticker'] = percent_below_df['Ticker'].str.replace("NSE:","")
ticker_below = percent_below_df["Ticker"].tolist()

In [13]:
import openpyxl

file_path = "C:\\Users\\sarda\\Dropbox\\TS Personal\\Trade Files\\Consolidated Trade File from 1st April 2022.xlsx"
def read_sheet_pivots(file_path, sheet_name):
    data = []
    workbook = openpyxl.load_workbook(file_path, read_only=True)
    sheet = workbook[sheet_name]
    
    row_number = 28
    column_m = 'M'
    column_n = 'N'
    
    
    while True:
        cell_m = sheet[column_m + str(row_number)].value
        cell_n = sheet[column_n + str(row_number)].value
        
        if cell_m is None and cell_n is None:
            break
        
        # Convert the cell values to strings and store them in the list
        if cell_m is not None:
            m_value = str(cell_m)
            if m_value != '0' and (not m_value.isdigit() or m_value.startswith('5')):
                data.append(m_value)
        
        if cell_n is not None:
            n_value = str(cell_n)
            if n_value != '0' and (not n_value.isdigit() or n_value.startswith('5')):
                data.append(n_value)
        
        row_number += 1
    
    return data

# Example usage

sheet_name = 'Pivots'
holding_list = read_sheet_pivots(file_path, sheet_name)
print(holding_list)


##saving the holdings to csv for uploading to chartink and trading view

# Specify the name of the CSV file you want to create
csv_file_name = output_path + "0 My_Holdings.csv"

# Write the data to the CSV file
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Company"])  # Optional: Write a header row
    writer.writerows([[item] for item in holding_list])


['UJJIVANSFB', 'REMSONSIND', 'MARINE', 'JMFINANCIL', 'BLKASHYAP', 'ICICIBANK', 'KSOLVES', 'MANKIND', 'ISGEC', 'L&tfh', 'INDIGOPNTS', 'MAGSON', 'KOTAKBANK', 'ARVINDFASN', 'PRIMESECU', '543597', 'EDELWEISS', 'HBLPOWER', 'MEDPLUS', 'EIHOTEL', '542753', 'GOCOLORS', 'CHEMPLASTS', '531968', 'MANPASAND']


In [14]:
# Remove duplicates and sort the list in-place
#my_list = list(set(my_list))
#my_list.sort()

list_below_1000 = ['522101','530129','538734','540358','540358','543953','543971','20microns','annapurna',
                   'arihantsup','chembond','dharmaj','global','indotech','kapston','kotharipet','magson',
                   'mblinfra','modisonltd','ndrauto','pyramid','semac','skygold','venusrem','walchannag']

list_below_1000 = list(set(list_below_1000))
list_below_1000.sort()


list_1k_10k = ['531112','542669','543620','anantraj','anup','apollo','arvind','arvindfasn','bajajhind','bancoindia',
               'bclind','bluejet','cignititec','cupid','dbrealty','diamondyd','dollar','electcast','emil','focus',
               'forcemot','gabriel','gensol','genuspower','hariompipe','hblpower','hcc','hirect','hitech','hpl',
               'icil','ifglexpor','iiflsec','ikio','inoxwind','jklakshmi','jktyre','jpassociat','jppower','kalamandir',
               'kesoramind','kfintech','kpigreen','ltfoods','lumaxtech','maninds','marine','moil','mvgjl','neulandlab',
               'newgen','paragmilk','peninland','pfs','pokarna','premexpln','prsmjohnsn','ramky','rategain','repcohome',
               'rpglife','sadhnaniq','sanghvimov','satin','sdbl','senco','sepc','shripiston','shrirampps','skipper',
               'somanycera','spandana','tci','tdpowersys','thomascook','tril','tv18brdcst','vijaya','zaggle',
               'zeemedia','awhcl','indramedco','marksans']

list_1k_10k = list(set(list_1k_10k))
list_1k_10k.sort()

list_above_10k = ['adanipower','amber','angelone','bikaji','bluestarco','brigade','bse','cdsl','cello','concordbio',
                  'creditacc','elecon','idea','jindalsaw','jkcement','jwl','kpittech','l&tfh','lalpathlab','lodha',
                  'mcx','patanjali','paytm','phoenixltd','piind','poonawalla','pplpharma','prestige','radico',
                  'recltd','rkforge','shriramfin','solarinds','sonatsoftw','suzlon','tatamotors','titagarh','trent',
                  'unionbank','welspunind','zomato','medanta','swanenergy','welcorp']

list_above_10k = list(set(list_above_10k))
list_above_10k.sort()


In [15]:
heading_non_fno = "          Top 50 7-day deliveries non_fno " + report_date
heading_sme = "          Top 50 7-day deliveries sme " + report_date

heading_3_percent = "          7-day deliveries up_3_percent or new_highs above 500 cr " + report_date
heading_3_percent_below = "          7-day deliveries up_3_percent or new_highs below 500 cr " + report_date

heading_3x = "          7-day deliveries 3x dely volume " + report_date
heading_holding = "          7-day deliveries for holdings " + report_date

heading_above_10k = "          7-day deliveries mktcap above 10k cr " + report_date
heading_below_1000 = "          7-day deliveries mktcap below 1000 cr " + report_date
heading_1k_10k = "            7-day deliveries mktcap 1k - 10k cr " + report_date

In [16]:
title_non_fno = "Non_fno top deliveries " + report_date + " with averages"
title_sme = "SME top deliveries " + report_date + " with averages"

title_3_percent = "         up_3_percent or new_highs above 500 cr " + report_date + " with averages"
title_3_percent_below = "         up_3_percent or new_highs below 500 cr " + report_date + " with averages"

title_3x = "          3x dely volume deliveries " + report_date + " with averages"
title_holding = "          7-day deliveries for holdings " + report_date + " with averages"

title_above_10k = "         Top deliveries mktcap above 10k cr " + report_date + " with averages"
title_below_1000 = "         Top deliveries mktcap below 1000 cr " + report_date + " with averages"
title_1k_10k = "          Top deliveries mktcap 1k - 10k cr " + report_date + " with averages"

In [17]:
print(heading_3_percent)
print("")
print_formatted_rows(ticker_list,title_3_percent)
print()
print()
print()
list_process_new(ticker_list, heading_3_percent)

          7-day deliveries up_3_percent or new_highs above 500 cr 231226

         up_3_percent or new_highs above 500 cr 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Indo-National Ltd.                 NIPPOBATRY    829.85    691.55       621      0.00      0.00      0.00      0.00
J Kumar Infraprojects Ltd.         JKIL          593.50    494.60     4,491      0.00      0.00      0.00      0.00
Themis Medicare Ltd.               THEMISMED     219.05    182.55     2,016      0.00      0.00      0.00      0.00
Gallantt Ispat Ltd.                GALLANTT      169.70    141.45     4,095      0.00      0.00      0.00      0.00
Syncom Formulations (India) Lt     SYNCOMF        14.15     11.80     1,330      0.00      0.00      0.00      0.00

Solara Active Pharma Sciences      SOLARA        375.95    329.05     1,349      0.00      0.00      0.00      0.00
Dynamatic Technologies Ltd.        DYNAM

Shriram Pistons & Rings Ltd        SHRIPISTON   1341.30   1277.45     5,908      0.00      0.00      0.00      0.00
D.P. Abhushan Ltd.                 DPABHUSHAN    625.10    595.35     1,391      0.00      0.00      0.00      0.00
Cupid Ltd.                         CUPID         967.90    921.85     1,291      0.00      0.00      0.00      0.00

V2 Retail Ltd.                     V2RETAIL      290.30    276.50     1,004      0.00      0.00      0.00      0.00
Black Box Ltd.                     BBOX          251.55    239.60     4,224      0.00      0.00      0.00      0.00
Walchandnagar Industries Ltd.      WALCHANNAG    196.95    187.60       905      0.00      0.00      0.00      0.00
Mufin Green Finance Ltd.           MUFIN         180.35    171.80     2,723      0.00      0.00      0.00      0.00
Om Infra Ltd.                      OMINFRAL      151.00    143.85     1,454      0.00      0.00      0.00      0.00

W.S. Industries (India) Ltd.       WSI           115.55    110.10     

NMDC Ltd.                          NMDC          201.85    195.40    59,155      0.00      0.00      0.00      0.00
CyberTech Systems And Software     CYBERTECH     196.65    190.40       560      0.00      0.00      0.00      0.00
KDDL Ltd.                          KDDL         2767.70   2681.15     3,496      0.00      0.00      0.00      0.00
Sukhjit Starch & Chemicals Ltd     SUKHJITS      512.65    496.65       804      0.00      0.00      0.00      0.00

Gandhi Special Tubes Ltd.          GANDHITUBE    763.10    739.45       925      0.00      0.00      0.00      0.00
Network 18 Media & Investments     NETWORK18      88.85     86.10     9,285      0.00      0.00      0.00      0.00
Reliance Industrial Infrastruc     RIIL         1348.60   1307.05     2,033      0.00      0.00      0.00      0.00
Shivalik Rasayan Ltd.              SHIVALIK      637.05    617.55     1,001      0.00      0.00      0.00      0.00
Chambal Fertilisers and Chemic     CHAMBLFERT    371.15    359.80    15


Company Name: Themis Medicare Ltd. (NSE Code: THEMISMED, BSE Code: 530199, Mktcap:2,016, Networth:364)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         417,174          27,518         444,692     219.05
   231222          48,078           4,078          52,156     182.55
   231221          39,861           3,118          42,979     181.60
   231220          45,435           6,256          51,691     171.95
   231219          23,614             623          24,237     180.55
   231218          23,477             543          24,020     183.50
   231215          25,748           1,285          27,033     183.80




Company Name: Gallantt Ispat Ltd. (NSE Code: GALLANTT, BSE Code: 532726, Mktcap:4,095, Networth:2,303)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       1,270,532          99,233       1,369,765     169.70
   


Company Name: Bombay Burmah Trading Corporat (NSE Code: BBTC, BSE Code: 501425, Mktcap:10,234, Networth:3,990)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         206,376          17,318         223,694    1468.70
   231222          14,112           1,403          15,515    1328.40
   231221          27,269             976          28,245    1328.95
   231220          36,101           3,904          40,005    1272.95
   231219          40,297           1,652          41,949    1345.70
   231218          59,450           3,046          62,496    1344.95
   231215          33,781           1,817          35,598    1379.10




Company Name: Balaji Amines Ltd. (NSE Code: BALAMINES, BSE Code: 530999, Mktcap:8,461, Networth:1,608)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         234,201          47,295         281,496    261

...

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       3,213,486         309,725       3,523,211     419.00
   231222       4,082,892          97,900       4,180,792     406.60
   231221       5,946,091         463,339       6,409,430     412.20
   231220       5,350,945         490,580       5,841,525     404.45
   231219       2,499,166         138,210       2,637,376     435.15
   231218       4,487,773          95,157       4,582,930     435.10
   231215       8,352,190         273,258       8,625,448     431.85




Company Name: Andhra Paper Ltd. (NSE Code: ANDHRAPAP, BSE Code: 502330, Mktcap:2,426, Networth:1,758)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         235,116          43,124         278,240     608.80
   231222          70,143           1,467          71,610     590.85
   231221          30,851           4,828          35,679     578.80
   231220         

   231215         167,658           7,583         175,241     584.85




Company Name: Balmer Lawrie & Company Ltd. (NSE Code: BALMLAWRIE, BSE Code: 523319, Mktcap:3,941, Networth:1,658)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       2,472,798         321,315       2,794,113     230.25
   231222       4,362,384         419,645       4,782,029     224.90
   231221         575,231          49,846         625,077     187.45
   231220       1,452,330         145,287       1,597,617     179.75
   231219         564,389          44,010         608,399     191.00
   231218       1,387,327         108,313       1,495,640     191.95
   231215       2,102,193         204,088       2,306,281     188.35




Company Name: Emkay Taps And Cutting Tools L (NSE Code: EMKAYTOOLS, BSE Code: 0, Mktcap:761, Networth:257)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Deliv


Company Name: Tata Coffee Ltd. (NSE Code: TATACOFFEE, BSE Code: 532301, Mktcap:5,609, Networth:1,855)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         353,896          13,772         367,668     300.15
   231222         250,957           8,040         258,997     294.75
   231221         356,560           8,658         365,218     291.00
   231220       1,093,566         103,041       1,196,607     290.90
   231219         644,590          25,523         670,113     287.65
   231218         206,740           7,669         214,409     283.00
   231215         644,907          61,229         706,136     283.80




Company Name: Marine Electricals (India) Ltd (NSE Code: MARINE, BSE Code: 0, Mktcap:1,289, Networth:196)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         409,251               0         409,251      99.50
  

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          61,498           6,299          67,797    1421.10
   231222         115,930          27,332         143,262    1405.80
   231221          57,525           2,373          59,898    1300.90
   231220          85,475          14,374          99,849    1286.80
   231219          92,558           7,266          99,824    1296.90
   231218          55,925          11,623          67,548    1295.40
   231215          52,977           1,578          54,555    1269.70




Company Name: Asian Energy Services Ltd. (NSE Code: ASIANENE, BSE Code: 530355, Mktcap:1,108, Networth:230)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          84,135          16,122         100,257     286.95
   231222          69,950          12,512          82,462     284.05
   231221         273,362         149,165         422,527     280.55
   231220   

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         158,001           9,124         167,125    1618.55
   231222         827,392          19,108         846,500    1618.85
   231221          60,206             908          61,114    1487.00
   231220          69,695           1,878          71,573    1459.95
   231219         106,943           5,502         112,445    1468.20
   231218         405,024           1,940         406,964    1471.80
   231215          62,309           4,308          66,617    1419.85




Company Name: Control Print Ltd. (NSE Code: CONTROLPR, BSE Code: 522295, Mktcap:1,606, Networth:306)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          19,993             714          20,707    1001.65
   231222          47,962           2,119          50,081    1002.40
   231221          20,091           2,358          22,449     994.85
   231220          

In [18]:
print_formatted_rows(dely_3x_list,heading_3x)
print()
print()
print()
list_process_new(dely_3x_list, heading_3x)

          7-day deliveries 3x dely volume 231226


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Motisons Jewellers Limited         MOTISONS      103.55     55.00         0      0.00      0.00      0.00      0.00
Suraj Estate Developers Limite     SURAJEST      334.50    360.00         0      0.00      0.00      0.00      0.00
Muthoot Microfin Limited           MUTHOOTMF     266.15    291.00         0      0.00      0.00      0.00      0.00
Simbhaoli Sugars Ltd.              SIMBHALS       33.60     28.05       138      0.00      0.00      0.00      0.00
JSW Holdings Ltd.                  JSWHL        5285.55   5052.20     5,860      0.00      0.00      0.00      0.00

Solara Active Pharma Sciences      SOLARA        375.95    329.05     1,349      0.00      0.00      0.00      0.00
Mahindra Lifespace Developers      MAHLIFE       540.40    532.35     8,370      0.00      0.00      0.00      0.00
Hindustan Organic 

PNC Infratech Ltd.                 PNCINFRA      351.50    345.30     9,026      0.00      0.00      0.00      0.00
Suprajit Engineering Ltd.          SUPRAJIT      364.60    370.70     5,040      0.00      0.00      0.00      0.00
SIRCA Paints India Ltd.            SIRCA         411.70    399.35     2,260      0.00      0.00      0.00      0.00
JK Cement Ltd.                     JKCEMENT     3765.40   3782.20    29,238      0.00      0.00      0.00      0.00

Andhra Paper Ltd.                  ANDHRAPAP     608.80    590.85     2,426      0.00      0.00      0.00      0.00
Fineotex Chemical Ltd.             FCL           369.40    349.85     4,093      0.00      0.00      0.00      0.00
Tega Industries Ltd.               TEGA         1175.05   1088.75     7,846      0.00      0.00      0.00      0.00
Minda Corporation Ltd.             MINDACORP     386.25    384.00     9,236      0.00      0.00      0.00      0.00
Sai Kalamandir                     KALAMANDIR    274.90    277.50     4

   231215               0         145,739         145,739      31.52




Company Name: Bombay Burmah Trading Corporat (NSE Code: BBTC, BSE Code: 501425, Mktcap:10,234, Networth:3,990)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         206,376          17,318         223,694    1468.70
   231222          14,112           1,403          15,515    1328.40
   231221          27,269             976          28,245    1328.95
   231220          36,101           3,904          40,005    1272.95
   231219          40,297           1,652          41,949    1345.70
   231218          59,450           3,046          62,496    1344.95
   231215          33,781           1,817          35,598    1379.10




Company Name: Dynamatic Technologies Ltd. (NSE Code: DYNAMATECH, BSE Code: 505242, Mktcap:3,519, Networth:588)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Deli

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         340,903          83,581         424,484     160.40
   231222          37,885           6,783          44,668     145.00
   231221          38,026           3,042          41,068     144.10
   231220          65,338          39,658         104,996     142.05
   231219          29,760           1,336          31,096     145.20
   231218          39,164          16,116          55,280     147.85
   231215          35,267          13,039          48,306     147.80




Company Name: Balaji Amines Ltd. (NSE Code: BALAMINES, BSE Code: 530999, Mktcap:8,461, Networth:1,608)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         234,201          47,295         281,496    2614.60
   231222          25,959           4,945          30,904    2365.20
   231221          44,638           1,910          46,548    2365.55
   231220        


Company Name: Deep Industries Ltd. (NSE Code: DEEPINDS, BSE Code: 543288, Mktcap:1,596, Networth:1,433)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         235,673          17,582         253,255     250.20
   231222          30,481           6,938          37,419     251.95
   231221          41,018           1,650          42,668     247.75
   231220          66,644          17,965          84,609     244.60
   231219          71,364           4,858          76,222     251.45
   231218          79,657          10,938          90,595     252.70
   231215          70,849           8,364          79,213     255.65




Company Name: India Pesticides Ltd. (NSE Code: IPL, BSE Code: 543311, Mktcap:4,273, Networth:802)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       1,466,964         329,567       1,796,531     369.60
   2312

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         284,607          60,429         345,036     940.25
   231222          58,878           3,389          62,267     893.50
   231221          93,153          19,898         113,051     889.95
   231220         224,794           8,422         233,216     881.45
   231219         110,307           5,905         116,212     906.05
   231218         126,909           6,351         133,260     913.00
   231215         103,059          14,894         117,953     915.65




Company Name: Hi-Tech Pipes Ltd. (NSE Code: HITECH, BSE Code: 543411, Mktcap:1,538, Networth:399)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       3,326,803         117,117       3,443,920     112.25
   231222         574,045          49,637         623,682      99.95
   231221         514,442          53,968         568,410      98.50
   231220         688,

   231215         875,070          36,381         911,451     253.10




Company Name: Allied Digital Services Ltd. (NSE Code: ADSL, BSE Code: 532875, Mktcap:765, Networth:556)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       1,177,608          81,189       1,258,797     137.75
   231222         256,383          13,598         269,981     126.55
   231221          97,638          36,646         134,284     120.20
   231220         233,490          11,901         245,391     119.60
   231219         141,940           7,186         149,126     124.95
   231218         112,498          28,666         141,164     125.90
   231215         401,827          52,032         453,859     127.35




Company Name: MTAR Technologies Ltd. (NSE Code: MTARTECH, BSE Code: 543270, Mktcap:6,688, Networth:661)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Pric

   231215          82,027          15,920          97,947     615.20




Company Name: Urja Global Ltd. (NSE Code: URJA, BSE Code: 526987, Mktcap:802, Networth:173)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226      11,129,740       4,191,809      15,321,549      15.25
   231222       3,165,338         571,937       3,737,275      14.00
   231221       3,537,940       1,516,235       5,054,175      13.85
   231220       7,224,092         990,229       8,214,321      13.30
   231219       4,918,160       1,795,248       6,713,408      14.35
   231218       7,603,811         856,361       8,460,172      14.75
   231215       7,417,505       2,213,591       9,631,096      15.25




Company Name: Onmobile Global Ltd. (NSE Code: ONMOBILE, BSE Code: 532944, Mktcap:1,186, Networth:657)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226   


Company Name: Nesco Ltd. (NSE Code: NESCO, BSE Code: 505355, Mktcap:6,372, Networth:2,097)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         158,598          16,544         175,142     905.55
   231222          46,662           4,830          51,492     863.20
   231221          56,066           1,671          57,737     864.45
   231220         174,750           7,591         182,341     846.20
   231219          77,752           8,332          86,084     859.30
   231218         136,122           3,925         140,047     864.40
   231215          61,195           2,474          63,669     830.30




Company Name: Balu Forge Industries Ltd. (NSE Code: 531112, BSE Code: 531112, Mktcap:2,575, Networth:430)

ema10 / ema20 / ema50 / ema200     246.74  243.97  234.44  182.8

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226               0         255,075         255,075     


Company Name: Abbott India Ltd. (NSE Code: ABBOTINDIA, BSE Code: 500488, Mktcap:47,865, Networth:3,109)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          15,027             147          15,174   22452.55
   231222           4,611             342           4,953   22604.00
   231221           4,384             298           4,682   22354.75
   231220          11,675             991          12,666   22135.15
   231219           7,288             110           7,398   22902.25
   231218           7,457             129           7,586   22780.30
   231215           7,584              99           7,683   22694.80




Company Name: Akshar Spintex Ltd. (NSE Code: AKSHAR, BSE Code: 541303, Mktcap:127, Networth:43)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226      17,302,989       3,022,872      20,325,861       5.10
   231222

In [19]:

print(heading_non_fno)
print("")
print_formatted_rows(unique_non_fno,title_non_fno)
print()
print()
print()
list_process_new(unique_non_fno, heading_non_fno)

          Top 50 7-day deliveries non_fno 231226

Non_fno top deliveries 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Indian Renewable Energy Dev Ag     IREDA         101.95    109.85    27,399      0.00      0.00      0.00      0.00
Mazagon Dock Shipbuilders Ltd.     MAZDOCK      2287.85   2121.40    46,195      0.00      0.00      0.00      0.00
Castrol India Ltd.                 CASTROLIND    182.55    162.55    18,017      0.00      0.00      0.00      0.00
Adani Green Energy Ltd.            ADANIGREEN   1600.20   1533.10   253,289      0.00      0.00      0.00      0.00
Indian Railway Finance Corpora     IRFC           97.40     97.75   127,287      0.00      0.00      0.00      0.00

Cochin Shipyard Ltd.               COCHINSHIP   1374.50   1301.40    18,061      0.00      0.00      0.00      0.00
JIO Financial Services             JIOFIN        234.70    236.75   149,048      0.00    

Hathway Cable & Datacom Ltd.       HATHWAY        22.10     22.05     3,915      0.00      0.00      0.00      0.00

DCW Ltd.                           DCW            57.80     54.85     1,707      0.00      0.00      0.00      0.00
SJVN Ltd.                          SJVN           91.55     89.70    36,029      0.00      0.00      0.00      0.00
Network 18 Media & Investments     NETWORK18      88.85     86.10     9,285      0.00      0.00      0.00      0.00
IRB Infrastructure Developers      IRB            41.85     41.65    25,249      0.00      0.00      0.00      0.00
Orient Green Power Company Ltd     GREENPOWER     21.95     22.35     2,154      0.00      0.00      0.00      0.00

National Fertilizers Ltd.          NFL            85.90     84.25     4,213      0.00      0.00      0.00      0.00
Ultracab (India) Ltd.              538706         27.83     24.30       266     24.50     23.17     20.75     18.94
IFCI Ltd.                          IFCI           28.15     28.40     

Company Name: Muthoot Microfin Limited (NSE Code: MUTHOOTMF, BSE Code: 544055, Mktcap:0, Networth:0)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       4,751,657         301,137       5,052,794     266.15




Company Name: Motisons Jewellers Limited (NSE Code: MOTISONS, BSE Code: 544053, Mktcap:0, Networth:0)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       8,948,938       3,762,613      12,711,551     103.55




Company Name: Varun Beverages Ltd. (NSE Code: VBL, BSE Code: 540180, Mktcap:162,202, Networth:6,489)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         945,848          23,745         969,593    1250.05
   231222       3,056,431          63,980       3,120,411    1238.60
   231221       1,384,895         103,634       1,488,529    


Company Name: Garden Reach Shipbuilders & En (NSE Code: GRSE, BSE Code: 542011, Mktcap:9,991, Networth:1,563)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       1,064,973          40,312       1,105,285     872.20
   231222          97,186           3,761         100,947     819.65
   231221         211,768          21,029         232,797     817.05
   231220         455,850          57,221         513,071     806.60
   231219         657,724          67,866         725,590     849.60
   231218         160,076          29,296         189,372     833.50
   231215         153,516          16,809         170,325     827.95




Company Name:  (NSE Code: SETFNIF50, BSE Code: 0, Mktcap:0, Networth:0)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       4,310,021               0       4,310,021     223.54
   231222       4,292,266  


Company Name: Balaji Amines Ltd. (NSE Code: BALAMINES, BSE Code: 530999, Mktcap:8,461, Networth:1,608)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         234,201          47,295         281,496    2614.60
   231222          25,959           4,945          30,904    2365.20
   231221          44,638           1,910          46,548    2365.55
   231220          65,564           3,256          68,820    2272.70
   231219          39,860          10,017          49,877    2412.80
   231218         104,697           4,262         108,959    2398.90
   231215         108,681           3,405         112,086    2345.35




Company Name: Jaiprakash Power Ventures Ltd. (NSE Code: JPPOWER, BSE Code: 532627, Mktcap:9,519, Networth:10,706)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226      39,262,987      11,818,390      51,081,377    


Company Name: Reliance Power Ltd. (NSE Code: RPOWER, BSE Code: 532939, Mktcap:8,602, Networth:12,827)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226      12,359,615       3,000,158      15,359,773      22.60
   231222      17,230,295       2,920,694      20,150,989      22.40
   231221      28,065,744       6,363,306      34,429,050      22.70
   231220      38,098,672       7,293,690      45,392,362      21.40
   231219      24,083,090       4,907,215      28,990,305      23.75
   231218      16,291,004       4,508,040      20,799,044      23.80
   231215      20,309,198       4,928,408      25,237,606      23.60




Company Name: Urja Global Ltd. (NSE Code: URJA, BSE Code: 526987, Mktcap:802, Networth:173)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226      11,129,740       4,191,809      15,321,549      15.25
   231222      

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       5,624,933         610,369       6,235,302      19.35
   231222       5,402,762         448,524       5,851,286      19.85
   231221       6,422,408       2,610,658       9,033,066      19.60
   231220       9,692,782       2,457,448      12,150,230      18.60
   231219       5,263,607         947,672       6,211,279      19.90
   231218       6,080,235       1,657,913       7,738,148      20.05
   231215      10,061,290       1,203,888      11,265,178      19.95




Company Name: Hathway Cable & Datacom Ltd. (NSE Code: HATHWAY, BSE Code: 533162, Mktcap:3,915, Networth:4,227)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       5,724,256         417,090       6,141,346      22.10
   231222       4,878,918         930,925       5,809,843      22.05
   231221       6,005,327         961,146       6,966,473      21.65
   231220

   231215       1,255,345          31,122       1,286,467      35.10




Company Name: Bajaj Hindusthan Sugar Ltd. (NSE Code: BAJAJHIND, BSE Code: 500032, Mktcap:3,542, Networth:4,230)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       3,130,775         972,756       4,103,531      27.75
   231222       5,576,402         303,719       5,880,121      27.90
   231221       4,852,866         927,277       5,780,143      28.15
   231220      10,344,258       1,933,084      12,277,342      27.35
   231219       5,857,654       1,443,380       7,301,034      29.15
   231218      25,465,972       2,008,040      27,474,012      29.70
   231215       4,802,734         808,462       5,611,196      28.00




Company Name: Union Bank Of India (NSE Code: UNIONBANK, BSE Code: 532477, Mktcap:86,874, Networth:79,381)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery

In [20]:
print(heading_sme)
print("")
print_formatted_rows(sme_list,title_sme)
print()
print()
print()
list_process_new(sme_list,heading_sme)

          Top 50 7-day deliveries sme 231226

SME top deliveries 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Droneacharya Aerial Innovation     543713        175.20    185.75       420    191.32    190.91    186.08    165.06
Kalyani Cast Tech Limited          544023        446.75    406.15       321    406.53    392.49    352.79    302.71
VishnuSurya Infra                  VISHNUINFR    445.50    428.40     1,096      0.00      0.00      0.00      0.00
Amic Forging Limited               544037        395.90    353.05       415    344.40    317.71    285.61    261.21
Insolation Energy Ltd.             543620        685.00    696.20     1,448    684.06    672.14    607.58    377.44

Basilic Fly Studio                 BASILIC       302.05    301.95       702    294.99    294.44    291.89    288.10
All E Technologies Ltd.            ALLETEC       235.50    256.75       476    249.41    248.40  

   231215          92,000               0          92,000     340.40




Company Name: Amic Forging Limited (NSE Code: 544037, BSE Code: 544037, Mktcap:415, Networth:20)

ema10 / ema20 / ema50 / ema200     344.4  317.71  285.61  261.21

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226               0         154,000         154,000     395.90
   231222               0          71,000          71,000     353.05
   231221               0          59,000          59,000     351.05
   231220               0         166,000         166,000     328.15
   231219               0         221,000         221,000     364.30
   231218               0         410,000         410,000     363.05
   231215               0          26,000          26,000     353.45




Company Name: Insolation Energy Ltd. (NSE Code: 543620, BSE Code: 543620, Mktcap:1,448, Networth:68)

ema10 / ema20 / ema50 / ema200     684.06  672.14  607.58  377.44

     Date    NSE_Delivery    BSE_Deliv

   231215          87,600               0          87,600     252.75




Company Name: Indo US Bio-Tech Ltd. (NSE Code: 541304, BSE Code: 541304, Mktcap:491, Networth:65)

ema10 / ema20 / ema50 / ema200     234.55  223.85  211.51  158.62

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226               0         129,500         129,500     243.10
   231222               0          37,500          37,500     239.00
   231221               0          21,500          21,500     240.90
   231220               0          27,500          27,500     234.40
   231219               0          52,000          52,000     249.65
   231218               0          19,500          19,500     239.65
   231215               0           8,000           8,000     238.60




Company Name: Sheetal Universal Limited (NSE Code: SHEETAL, BSE Code: 0, Mktcap:99, Networth:7)

ema10 / ema20 / ema50 / ema200     72.42  73.21  74.27  75.08

     Date    NSE_Delivery    BSE_Delivery  To

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         230,000               0         230,000      86.55
   231222         120,000               0         120,000      80.10
   231221          60,000               0          60,000      81.95
   231220          90,000               0          90,000      82.15
   231219         140,000               0         140,000      85.35
   231218         144,000               0         144,000      84.35
   231215          30,000               0          30,000      89.05




Company Name: Supershakti Metaliks Ltd. (NSE Code: 541701, BSE Code: 541701, Mktcap:530, Networth:245)

ema10 / ema20 / ema50 / ema200     469.52  469.63  442.84  367.48

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226               0          43,200          43,200     460.00




Company Name: Accent Microcell Limited (NSE Code: ACCENTMIC, BSE Code: 0, Mktcap:588, Networth:44)

ema10 / ema20 / ema50 / ema200 

   231215         325,200               0         325,200     170.85




Company Name: Crop Life Science Limited (NSE Code: CLSL, BSE Code: 0, Mktcap:82, Networth:64)

ema10 / ema20 / ema50 / ema200     42.16  41.39  41.33  43.3

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         262,000               0         262,000      47.55
   231222          30,000               0          30,000      42.40
   231221          12,000               0          12,000      40.95
   231220          92,000               0          92,000      41.70
   231219          92,000               0          92,000      42.00
   231218          22,000               0          22,000      39.85
   231215          24,000               0          24,000      39.40




Company Name: Sahana System Ltd. (NSE Code: SAHANA, BSE Code: 0, Mktcap:518, Networth:41)

ema10 / ema20 / ema50 / ema200     594.64  577.04  501.24  308.95

     Date    NSE_Delivery    BSE_Delivery  Total_Deliver

   231215          11,400               0          11,400     134.00





In [21]:
print_formatted_rows(holding_list,title_holding)
print()
print()
print()
list_process_new(holding_list, heading_holding)

          7-day deliveries for holdings 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Ujjivan Small Finance Bank Ltd     UJJIVANSFB     57.10     57.70    11,174      0.00      0.00      0.00      0.00
Remsons Industries Ltd.            REMSONSIND    640.00    644.90       367      0.00      0.00      0.00      0.00
Marine Electricals (India) Ltd     MARINE         99.50     97.75     1,289      0.00      0.00      0.00      0.00
JM Financial Ltd.                  JMFINANCIL     96.80     97.45     9,239      0.00      0.00      0.00      0.00
B.L. Kashyap and Sons Ltd.         BLKASHYAP      65.40     64.95     1,479      0.00      0.00      0.00      0.00

ICICI Bank Ltd.                    ICICIBANK     995.10    994.30   698,288      0.00      0.00      0.00      0.00
KSolves India Ltd.                 KSOLVES      1204.50   1232.70     1,438      0.00      0.00      0.00      0.00
Mankin


Company Name: L&T Finance Holdings Ltd. (NSE Code: L&TFH, BSE Code: 533519, Mktcap:38,048, Networth:22,185)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         988,180          73,331       1,061,511     153.00
   231222       2,565,966         102,295       2,668,261     152.80
   231221       2,123,954          97,154       2,221,108     151.55
   231220       3,030,341         266,795       3,297,136     150.10
   231219       2,197,253          60,245       2,257,498     159.50
   231218       2,231,999          54,797       2,286,796     159.60
   231215       4,702,636         114,558       4,817,194     161.25




Company Name: Indigo Paints Ltd. (NSE Code: INDIGOPNTS, BSE Code: 543258, Mktcap:6,979, Networth:821)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          13,156           3,846          17,002    1468.55


Company Name: IITL Projects Ltd. (NSE Code: 531968, BSE Code: 531968, Mktcap:0, Networth:0)

ema10 / ema20 / ema50 / ema200     16.44  16.67  16.97  17.3

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231220               0             417             417      16.29
   231219               0           1,008           1,008      16.34
   231218               0           1,200           1,200      16.35



No data found for MANPASAND



In [22]:
print(heading_3_percent_below)
print("")
print_formatted_rows(ticker_below,title_3_percent_below)
print()
print()
print()
list_process_new(ticker_below, heading_3_percent_below)

          7-day deliveries up_3_percent or new_highs below 500 cr 231226

         up_3_percent or new_highs below 500 cr 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Brooks Laboratories Ltd.           BROOKS        162.50    135.45       427      0.00      0.00      0.00      0.00
VL E-Governance & IT Solutions     VLEGOV         41.60     34.70       441      0.00      0.00      0.00      0.00
Simbhaoli Sugars Ltd.              SIMBHALS       33.60     28.05       138      0.00      0.00      0.00      0.00
Wonder Electricals Ltd.            WEL           368.60    308.15       495      0.00      0.00      0.00      0.00
Starteck Finance Ltd.              STARTECK      410.00    343.60       409      0.00      0.00      0.00      0.00

Shah Metacorp Ltd.                 SHAH            5.25      4.60       220      0.00      0.00      0.00      0.00
Sikko Industries Ltd.              SIKKO

Shree Vasu Logistics Ltd.          SVLL          184.50    177.00       212      0.00      0.00      0.00      0.00
Tainwala Chemicals & Plastics      TAINWALCHM    149.55    143.50       139      0.00      0.00      0.00      0.00
Reliance Home Finance Ltd.         RHFL            3.75      3.60       183      0.00      0.00      0.00      0.00
Trigyn Technologies Ltd.           TRIGYN        140.65    135.05       434      0.00      0.00      0.00      0.00

ROX Hitech Limited                 ROXHITECH     165.70    159.15       378    165.07    157.77    149.68    143.97
The Motor & General Finance Lt     MOTOGENFIN     34.50     33.15       129      0.00      0.00      0.00      0.00
Shah Alloys Ltd.                   SHAHALLOYS     69.60     66.90       138      0.00      0.00      0.00      0.00
RKEC Projects Ltd.                 RKEC           74.90     72.00       180      0.00      0.00      0.00      0.00
SVP Global Textiles Ltd.           SVPGLOB        10.60     10.20      

   231215       1,327,847         549,486       1,877,333       3.30




Company Name: Sikko Industries Ltd. (NSE Code: SIKKO, BSE Code: 0, Mktcap:191, Networth:26)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         293,352               0         293,352     113.60
   231222         444,122               0         444,122     101.00
   231221         309,149               0         309,149      88.50
   231220          88,291               0          88,291      73.75
   231219          81,748               0          81,748      80.75
   231218         322,572               0         322,572      80.60
   231215         321,525               0         321,525      77.75




Company Name: Arabian Petroleum (NSE Code: ARABIAN, BSE Code: 0, Mktcap:114, Networth:25)

ema10 / ema20 / ema50 / ema200     98.5  97.47  91.16  81.18

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226


Company Name: SPL Industries Ltd. (NSE Code: SPLIL, BSE Code: 532651, Mktcap:207, Networth:193)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         113,327          24,219         137,546      71.30
   231222          24,737           1,438          26,175      67.20
   231221          30,400           2,042          32,442      67.60
   231220          61,811          17,042          78,853      65.90
   231219          22,287           5,761          28,048      68.65
   231218          31,154           1,410          32,564      68.80
   231215          27,160           1,901          29,061      69.15




Company Name: Oricon Enterprises Ltd. (NSE Code: ORICONENT, BSE Code: 513121, Mktcap:513, Networth:1,131)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         605,307         131,828         737,135      32.80
   2312

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226           6,894               0           6,894     246.85
   231222           5,018               0           5,018     235.10
   231221           2,829               0           2,829     226.00
   231220           4,355               0           4,355     225.35
   231219           1,555               0           1,555     228.90
   231218           3,547               0           3,547     228.35
   231215           3,847               0           3,847     228.80




Company Name: Premier Polyfilm Ltd. (NSE Code: PREMIERPOL, BSE Code: 514354, Mktcap:343, Networth:83)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          15,840           1,679          17,519     163.90
   231222          16,304           1,593          17,897     156.10
   231221          11,113           1,843          12,956     148.70
   231220         


Company Name: GP Petroleums Ltd. (NSE Code: GULFPETRO, BSE Code: 532543, Mktcap:269, Networth:290)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          86,348          51,220         137,568      52.85
   231222          45,194          10,154          55,348      50.35
   231221          46,755           7,689          54,444      51.55
   231220         116,747          18,456         135,203      50.05
   231219          39,943          14,458          54,401      52.55
   231218          46,725           4,942          51,667      52.70
   231215          41,675           5,387          47,062      52.95




Company Name: BPL Ltd. (NSE Code: BPL, BSE Code: 500074, Mktcap:425, Networth:248)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          86,530          37,808         124,338      86.75
   231222          46,410  

   231215               0          99,422          99,422       7.75




Company Name: Prakash Steelage Ltd. (NSE Code: PRAKASHSTL, BSE Code: 533239, Mktcap:163, Networth:-28)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         485,128         439,284         924,412       9.25
   231222         984,248         810,077       1,794,325       8.85
   231221       2,477,433       1,940,476       4,417,909       8.45
   231220       5,510,764       2,641,801       8,152,565       7.70
   231219       2,706,880       1,268,275       3,975,155       7.70
   231218         883,121         475,775       1,358,896       7.00
   231215       1,041,645         277,751       1,319,396       6.80




Company Name: Ducon Infratechnologies Ltd. (NSE Code: DUCON, BSE Code: 534674, Mktcap:243, Networth:120)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Pric


Company Name: The Motor & General Finance Lt (NSE Code: MOTOGENFIN, BSE Code: 501343, Mktcap:129, Networth:74)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226           8,557           3,844          12,401      34.50
   231222          10,525           4,942          15,467      33.15
   231221           3,050           5,261           8,311      32.20
   231220          15,123             417          15,540      32.00
   231219           6,369           5,861          12,230      32.75
   231218          13,921             995          14,916      32.50
   231215           4,948           1,460           6,408      32.35




Company Name: Shah Alloys Ltd. (NSE Code: SHAHALLOYS, BSE Code: 513436, Mktcap:138, Networth:32)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          78,031          32,657         110,688      69.60
 


Company Name: Global Vectra Helicorp Ltd. (NSE Code: GLOBALVECT, BSE Code: 532773, Mktcap:157, Networth:15)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          42,430           3,645          46,075     110.80
   231222          17,974           4,372          22,346     106.80
   231221          23,205           1,517          24,722     105.25
   231220          58,594           4,657          63,251     103.75
   231219          95,974          13,465         109,439     108.25
   231218          97,117          15,470         112,587     114.45
   231215          38,104          14,120          52,224     110.30




Company Name: Vipul Ltd. (NSE Code: VIPULLTD, BSE Code: 511726, Mktcap:220, Networth:82)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          92,988          12,019         105,007      18.25
   231222   

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         546,048         173,592         719,640      22.75
   231218         461,389         241,078         702,467      23.95




Company Name: Rocking Deals Circular Economy (NSE Code: ROCKINGDCE, BSE Code: 0, Mktcap:177, Networth:0)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          71,000               0          71,000     312.65
   231222         229,000               0         229,000     331.90
   231221          25,000               0          25,000     276.60
   231220          43,000               0          43,000     261.30
   231219          48,000               0          48,000     279.35
   231218          36,000               0          36,000     293.00
   231215          33,000               0          33,000     280.00





In [23]:
print_formatted_rows(list_above_10k,title_above_10k)
print()
print()
print()
list_process_new(list_above_10k, heading_above_10k)

         Top deliveries mktcap above 10k cr 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Adani Power Ltd.                   ADANIPOWER    511.70    512.55   197,823      0.00      0.00      0.00      0.00
Amber Enterprises India Ltd.       AMBER        3041.15   3051.55    10,245      0.00      0.00      0.00      0.00
Angel One Ltd.                     ANGELONE     3288.20   3292.35    27,611      0.00      0.00      0.00      0.00
Bikaji Foods International Ltd     BIKAJI        553.30    552.35    13,823      0.00      0.00      0.00      0.00
Blue Star Ltd.                     BLUESTARCO    946.95    951.25    19,465      0.00      0.00      0.00      0.00

Brigade Enterprises Ltd.           BRIGADE       859.50    868.80    19,992      0.00      0.00      0.00      0.00
BSE Ltd.                           BSE          2310.60   2319.90    31,280      0.00      0.00      0.00      0.00
Ce

   231215         672,244               0         672,244    1890.60




Company Name: Cello World Limited (NSE Code: CELLO, BSE Code: 544012, Mktcap:16,257, Networth:977)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         181,371           2,161         183,532     765.50
   231222         192,695           6,445         199,140     769.20
   231221         130,301           4,380         134,681     775.10
   231220         160,917          22,515         183,432     762.40
   231219          82,270           3,355          85,625     787.60
   231218         184,922           6,717         191,639     799.80
   231215         219,033           7,421         226,454     791.45




Company Name: Concord Biotech (NSE Code: CONCORDBIO, BSE Code: 543960, Mktcap:15,715, Networth:1,354)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   23


Company Name: Macrotech Developers Ltd. (NSE Code: LODHA, BSE Code: 543287, Mktcap:91,717, Networth:12,989)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         146,469          13,987         160,456     952.80
   231222         271,104          24,712         295,816     937.20
   231221         393,013          12,117         405,130     923.35
   231220         356,317           5,302         361,619     918.70
   231219         588,273          19,726         607,999     920.90
   231218         329,335           7,212         336,547     930.00
   231215         267,683           4,208         271,891     934.85




Company Name: Multi Commodity Exchange Of In (NSE Code: MCX, BSE Code: 534091, Mktcap:16,043, Networth:1,331)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          60,607           2,839          63,446   

   231215       8,352,190         273,258       8,625,448     431.85




Company Name: Ramkrishna Forgings Ltd. (NSE Code: RKFORGE, BSE Code: 532527, Mktcap:13,026, Networth:1,545)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         132,833           3,067         135,900     724.65
   231222         143,609           3,524         147,133     724.40
   231221         141,502           4,677         146,179     723.35
   231220         196,194          19,743         215,937     715.55
   231219         152,773          10,232         163,005     753.70
   231218         371,988           3,800         375,788     757.65
   231215         141,482           4,341         145,823     760.70




Company Name: Shriram Finance Ltd. (NSE Code: SHRIRAMFIN, BSE Code: 511218, Mktcap:76,297, Networth:46,307)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery  

No data found for WELSPUNIND


Company Name: Zomato Ltd. (NSE Code: ZOMATO, BSE Code: 543320, Mktcap:108,929, Networth:19,742)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226      12,949,042         387,105      13,336,147     125.00
   231222      22,770,017       1,407,284      24,177,301     128.50
   231221      17,447,744       1,006,570      18,454,314     127.35
   231220      21,341,773         659,155      22,000,928     124.75
   231219      22,941,383       2,240,154      25,181,537     130.25
   231218      38,551,889       3,871,643      42,423,532     128.55
   231215      24,157,585         401,990      24,559,575     123.60





In [24]:
print_formatted_rows(list_1k_10k, title_1k_10k)
print()
print()
print()

list_process_new(list_1k_10k, heading_1k_10k)

          Top deliveries mktcap 1k - 10k cr 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Balu Forge Industries Ltd.         531112        250.95    248.20     2,575    246.74    243.97    234.44    182.80
BMW Industries Ltd.                542669         54.67     52.45     1,231     54.90     54.94     51.43     41.25
Insolation Energy Ltd.             543620        685.00    696.20     1,448    684.06    672.14    607.58    377.44
Anant Raj Ltd.                     ANANTRAJ      299.05    303.15     9,713      0.00      0.00      0.00      0.00
The Anup Engineering Ltd.          ANUP         2732.00   2638.00     2,715      0.00      0.00      0.00      0.00

Apollo Micro Systems Ltd.          APOLLO        114.05    114.70     2,987      0.00      0.00      0.00      0.00
Arvind Ltd.                        ARVIND        266.35    259.30     6,952      0.00      0.00      0.00      0.00
Ar

Skipper Ltd.                       SKIPPER       224.95    225.85     2,315      0.00      0.00      0.00      0.00
Somany Ceramics Ltd.               SOMANYCERA    716.95    726.75     3,027      0.00      0.00      0.00      0.00
Spandana Sphoorty Financial Lt     SPANDANA     1029.70   1036.55     7,347      0.00      0.00      0.00      0.00
Transport Corporation Of India     TCI           818.15    816.75     6,352      0.00      0.00      0.00      0.00

TD Power Systems Ltd.              TDPOWERSYS    266.00    267.55     4,141      0.00      0.00      0.00      0.00
Thomas Cook (India) Ltd.           THOMASCOOK    140.30    139.80     6,576      0.00      0.00      0.00      0.00
Transformers & Rectifiers (Ind     TRIL          217.40    224.05     3,117      0.00      0.00      0.00      0.00
TV18 Broadcast Ltd.                TV18BRDCST     52.15     51.00     8,947      0.00      0.00      0.00      0.00
Vijaya Diagnostic Centre Ltd.      VIJAYA        630.80    640.90     6

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       3,130,775         972,756       4,103,531      27.75
   231222       5,576,402         303,719       5,880,121      27.90
   231221       4,852,866         927,277       5,780,143      28.15
   231220      10,344,258       1,933,084      12,277,342      27.35
   231219       5,857,654       1,443,380       7,301,034      29.15
   231218      25,465,972       2,008,040      27,474,012      29.70
   231215       4,802,734         808,462       5,611,196      28.00




Company Name: Banco Products (India) Ltd. (NSE Code: BANCOINDIA, BSE Code: 500039, Mktcap:4,642, Networth:1,047)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         212,635          40,131         252,766     651.05
   231222         264,070          24,866         288,936     618.75
   231221          86,063           8,831          94,894     583.10
   2312

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          35,191           4,064          39,255     767.05
   231222          15,359           5,495          20,854     768.80
   231221          26,879           3,077          29,956     769.65
   231220          46,990          18,122          65,112     761.65
   231219          38,060           4,799          42,859     792.05
   231218          32,451          12,250          44,701     799.15
   231215          61,345          10,948          72,293     795.05




Company Name: Genus Power Infrastructures Lt (NSE Code: GENUSPOWER, BSE Code: 530343, Mktcap:5,864, Networth:1,169)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         143,420          25,054         168,474     228.05
   231222         139,711          27,071         166,782     231.15
   231221         218,267          76,941         295,208     232.90
   2


Company Name: IKIO Lighting Ltd. (NSE Code: IKIO, BSE Code: 543923, Mktcap:2,594, Networth:524)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         216,912          12,128         229,040     335.80
   231222          94,040          17,854         111,894     342.90
   231221          82,621          31,299         113,920     341.85
   231220         217,677          50,620         268,297     338.25
   231219         180,451          10,228         190,679     356.35
   231218         130,063           6,529         136,592     350.80
   231215          96,882          20,513         117,395     353.65




Company Name: Indraprastha Medical Corporati (NSE Code: INDRAMEDCO, BSE Code: 532150, Mktcap:1,645, Networth:415)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          80,918          15,990          96,908     178.65

   231215          75,236           9,535          84,771     384.20




Company Name: Man Industries (India) Ltd. (NSE Code: MANINDS, BSE Code: 513269, Mktcap:1,508, Networth:1,172)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         188,516          58,056         246,572     251.85
   231222         281,962          24,208         306,170     251.10
   231221         196,532          15,473         212,005     252.45
   231220         277,114          78,997         356,111     245.55
   231219         128,544           8,033         136,577     267.60
   231218          98,213          19,792         118,005     273.45
   231215          84,246          11,433          95,679     277.45




Company Name: Marine Electricals (India) Ltd (NSE Code: MARINE, BSE Code: 0, Mktcap:1,289, Networth:196)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery   


Company Name: RateGain Travel Technologies L (NSE Code: RATEGAIN, BSE Code: 543417, Mktcap:8,699, Networth:771)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         178,049          14,572         192,621     730.10
   231222         408,435          51,706         460,141     740.20
   231221         376,784         147,860         524,644     735.35
   231220         243,963          41,340         285,303     693.35
   231219         163,762          15,802         179,564     716.85
   231218         707,513          50,586         758,099     717.40
   231215         372,005           9,762         381,767     696.90




Company Name: Repco Home Finance Ltd. (NSE Code: REPCOHOME, BSE Code: 535322, Mktcap:2,482, Networth:2,686)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          33,378           1,856          35,234 

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226         119,909           5,266         125,175     224.95
   231222         197,715          18,653         216,368     225.85
   231221         204,559          13,939         218,498     219.60
   231220         709,239          26,938         736,177     214.70
   231219          93,777          46,456         140,233     233.25
   231218         131,252          20,092         151,344     232.80
   231215         138,364          20,558         158,922     232.10




Company Name: Somany Ceramics Ltd. (NSE Code: SOMANYCERA, BSE Code: 531548, Mktcap:3,027, Networth:820)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          20,362           1,759          22,121     716.95
   231222          12,317           1,210          13,527     726.75
   231221          11,622           3,188          14,810     708.90
   231220       

In [25]:
print_formatted_rows(list_below_1000, title_below_1000)
print()
print()
print()
list_process_new(list_below_1000, heading_below_1000)

         Top deliveries mktcap below 1000 cr 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


20 Microns Ltd.                    20MICRONS     188.95    189.45       667      0.00      0.00      0.00      0.00
Kilburn Engineering Ltd.           522101        281.70    280.80     1,067    282.14    278.92    255.32    179.65
Nile Ltd.                          530129        874.75    882.10       262    878.40    881.02    844.51    735.64
Ceinsys Tech Ltd.                  538734        351.45    356.30       542    356.46    356.08    336.10    251.65
RMC Switchgears Ltd.               540358        587.75    578.75       608    584.15    586.85    603.08    515.17

Khazanchi Jewellers Ltd            543953        274.00    273.15       678    274.50    276.12    264.36    207.34
Bondada Engineering Limited        543971        379.75    391.75       817    395.87    397.40    359.02    242.00
A


Company Name: Annapurna Swadisht Ltd. (NSE Code: ANNAPURNA, BSE Code: 0, Mktcap:556, Networth:93)

ema10 / ema20 / ema50 / ema200     331.49  334.41  326.04  278.64

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          26,000               0          26,000     319.50
   231222          41,000               0          41,000     323.80
   231221          23,500               0          23,500     330.20
   231220          28,500               0          28,500     322.75
   231219          41,000               0          41,000     330.00
   231218          70,500               0          70,500     326.40
   231215          21,000               0          21,000     338.35




Company Name: Arihant Superstructures Ltd. (NSE Code: ARIHANTSUP, BSE Code: 506194, Mktcap:1,118, Networth:234)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          40,156           1,082       

   231215             990           1,459           2,449     728.00




Company Name: Pyramid Technoplast Limited (NSE Code: PYRAMID, BSE Code: 543969, Mktcap:722, Networth:208)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          79,850           7,611          87,461     195.85
   231222          57,671          13,027          70,698     192.90
   231221          59,880           6,548          66,428     192.35
   231220         151,386          14,988         166,374     191.30
   231219         263,150          10,915         274,065     199.25
   231218         302,945          41,136         344,081     195.65
   231215          54,724          11,012          65,736     191.40




Company Name: Semac Consultants Ltd (NSE Code: SEMAC, BSE Code: 505368, Mktcap:708, Networth:110)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
  

In [26]:
print("                         Top Deliveries with averages ", report_date)
print('________________________________________________________________________________________________________________')
print("")
print("")
print("")

print_formatted_rows(unique_non_fno,title_non_fno)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(sme_list,title_sme)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(list_above_10k,title_above_10k)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(list_1k_10k,title_1k_10k)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(list_below_1000, title_below_1000)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()


                         Top Deliveries with averages  231226
________________________________________________________________________________________________________________



Non_fno top deliveries 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Indian Renewable Energy Dev Ag     IREDA         101.95    109.85    27,399      0.00      0.00      0.00      0.00
Mazagon Dock Shipbuilders Ltd.     MAZDOCK      2287.85   2121.40    46,195      0.00      0.00      0.00      0.00
Castrol India Ltd.                 CASTROLIND    182.55    162.55    18,017      0.00      0.00      0.00      0.00
Adani Green Energy Ltd.            ADANIGREEN   1600.20   1533.10   253,289      0.00      0.00      0.00      0.00
Indian Railway Finance Corpora     IRFC           97.40     97.75   127,287      0.00      0.00      0.00      0.00

Cochin Shipyard Ltd.               COCHINSHIP   1374.50   1301.40    18,061  

SJVN Ltd.                          SJVN           91.55     89.70    36,029      0.00      0.00      0.00      0.00
Network 18 Media & Investments     NETWORK18      88.85     86.10     9,285      0.00      0.00      0.00      0.00
IRB Infrastructure Developers      IRB            41.85     41.65    25,249      0.00      0.00      0.00      0.00
Orient Green Power Company Ltd     GREENPOWER     21.95     22.35     2,154      0.00      0.00      0.00      0.00

National Fertilizers Ltd.          NFL            85.90     84.25     4,213      0.00      0.00      0.00      0.00
Ultracab (India) Ltd.              538706         27.83     24.30       266     24.50     23.17     20.75     18.94
IFCI Ltd.                          IFCI           28.15     28.40     7,003      0.00      0.00      0.00      0.00
CESC Ltd.                          CESC          123.40    119.15    16,357      0.00      0.00      0.00      0.00
Trident Ltd.                       TRIDENT        36.05     36.15    18

Central Depository Services (I     CDSL         1851.05   1836.95    19,343      0.00      0.00      0.00      0.00
Cello World Limited                CELLO         765.50    769.20    16,257      0.00      0.00      0.00      0.00
Concord Biotech                    CONCORDBIO   1506.10   1504.70    15,715      0.00      0.00      0.00      0.00

CreditAccess Grameen Ltd.          CREDITACC    1613.25   1627.50    25,711      0.00      0.00      0.00      0.00
Elecon Engineering Company Ltd     ELECON        940.25    893.50    10,549      0.00      0.00      0.00      0.00
Vodafone Idea Ltd.                 IDEA           13.35     13.60    65,037      0.00      0.00      0.00      0.00
Jindal Saw Ltd.                    JINDALSAW     422.15    424.40    13,481      0.00      0.00      0.00      0.00
JK Cement Ltd.                     JKCEMENT     3765.40   3782.20    29,238      0.00      0.00      0.00      0.00

Jupiter Wagons Ltd.                JWL           320.30    323.70    1

Indraprastha Medical Corporati     INDRAMEDCO    178.65    179.55     1,645      0.00      0.00      0.00      0.00

Inox Wind Ltd.                     INOXWIND      466.15    453.25    15,183      0.00      0.00      0.00      0.00
JK Lakshmi Cement Ltd.             JKLAKSHMI     885.55    870.05    10,372      0.00      0.00      0.00      0.00
JK Tyre & Industries Ltd.          JKTYRE        377.40    383.30     9,290      0.00      0.00      0.00      0.00
Jaiprakash Associates Ltd.         JPASSOCIAT     20.85     21.10     5,118      0.00      0.00      0.00      0.00
Jaiprakash Power Ventures Ltd.     JPPOWER        13.95     13.25     9,519      0.00      0.00      0.00      0.00

Sai Kalamandir                     KALAMANDIR    274.90    277.50     4,219      0.00      0.00      0.00      0.00
Kesoram Industries Ltd.            KESORAMIND    168.95    167.05     5,241      0.00      0.00      0.00      0.00
KFin Technologies Ltd.             KFINTECH      502.60    491.85     

Pyramid Technoplast Limited        PYRAMID       195.85    192.90       722      0.00      0.00      0.00      0.00

Semac Consultants Ltd              SEMAC        2303.00   2350.00       708      0.00      0.00      0.00      0.00
Sky Gold Ltd.                      SKYGOLD      1003.90    956.10     1,079      0.00      0.00      0.00      0.00
Venus Remedies Ltd.                VENUSREM      395.80    389.95       527      0.00      0.00      0.00      0.00
Walchandnagar Industries Ltd.      WALCHANNAG    196.95    187.60       905      0.00      0.00      0.00      0.00
________________________________________________________________________________________________________________
________________________________________________________________________________________________________________



In [27]:
check_stock = ['inspirisys','coalindia','wockpharma', 'imagicaa','stltech','cub','sagcem','sapphire']
list_process_new(check_stock, "temp check")

temp check

Company Name: Inspirisys Solutions Ltd. (NSE Code: INSPIRISYS, BSE Code: 532774, Mktcap:338, Networth:-9)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226          30,507           5,660          36,167      89.20
   231222         168,419          25,960         194,379      89.50
   231221          33,247          11,601          44,848      93.45
   231220          71,847           7,565          79,412      89.20
   231219          38,058          11,099          49,157      87.65
   231218          12,577           8,592          21,169      83.50
   231215           6,449           6,229          12,678      81.00




Company Name: Coal India Ltd. (NSE Code: COALINDIA, BSE Code: 533278, Mktcap:225,618, Networth:69,686)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231226       5,589,977         461,039       6,051,016 

In [28]:

import fpdf

data=['tushar', 'sarda', 'test', 'new' ]

pdf = fpdf.FPDF(format='letter')
pdf.add_page()
pdf.set_font("Arial", size=12)

for i in data:
    pdf.write(5,str(i))
    pdf.ln()
pdf.output("testings.pdf")




''

In [29]:
import fpdf

def list_process_to_pdf(list_name, header):
    # Create PDF instance with A4 format
    pdf = fpdf.FPDF(format='A4')
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Add header to PDF
    pdf.cell(200, 10, txt=header, ln=True, align='C')
    pdf.ln(1)  # Set line break to 1

    for company in list_name:
        if company.isdigit():
            company_int = int(company)
            company_data = combined_df[combined_df['BSE_Code'] == company_int]
        else:
            company = company.upper()
            company_data = combined_df[combined_df['NSE_Code'] == company]

        if not company_data.empty:
            company_name = company_data['Company Name'].values[0]
            nse_code = company_data['NSE_Code'].values[0]
            bse_code = company_data['BSE_Code'].values[0]

            ema10, ema20, ema50, ema200 = averages(nse_code)
            ema_str = "ema10 / ema20 / ema50 / ema200"

            mktcap_value = get_mktcap(nse_code)

            # Add company information to PDF
            pdf.cell(0, 8, txt=f"Company Name: {company_name}", ln=True)
            pdf.cell(0, 8, txt=f"(NSE Code: {nse_code}, BSE Code: {bse_code}, Mktcap: {mktcap_value[0]:,.0f})", ln=True)
            pdf.cell(0, 8, txt=f"{ema_str}", ln=True)
            pdf.cell(0, 8, txt=f"{str(ema10[0]):<10}  {str(ema20[0]):<10}  {str(ema50[0]):<10}  {str(ema200[0]):<10}", ln=True)
            pdf.ln(1)  # Set line break to 1

            formatted_columns = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
            formatted_data = company_data[formatted_columns].copy()

            # Apply alignment to specific columns in formatted_data
            formatted_data['Date'] = formatted_data['Date'].astype(str).apply(lambda x: x.ljust(12))
            formatted_data['NSE_Delivery'] = formatted_data['NSE_Delivery'].apply(lambda x: f"{int(x):,}".rjust(15))
            formatted_data['BSE_Delivery'] = formatted_data['BSE_Delivery'].apply(lambda x: f"{int(x):,}".rjust(15))
            formatted_data['Total_Delivery'] = formatted_data['Total_Delivery'].apply(lambda x: f"{int(x):,}".rjust(15))
            formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{x:.2f}".rjust(15))

            # Add formatted_data to PDF
            formatted_data_str = formatted_data.to_string(index=False)
            pdf.multi_cell(0, 8, txt=formatted_data_str)
            pdf.cell(0, 8, txt="\n" + "=" * 50 + "\n", ln=True)  # Set line break to 1

        else:
            pdf.cell(0, 8, txt=f"No data found for {company}\n", ln=True)

    # Output the PDF to a file
    pdf.output("output_a4_formatted_data_alignment.pdf")

# Example usage:
#check_stock = ['ABC', 'XYZ']
list_process_to_pdf(check_stock, "Temp Check")


In [30]:
import fpdf

def list_process_to_pdf(list_name, header):
    # Create PDF instance with A4 format
    pdf = fpdf.FPDF(format='A4')
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Add header to PDF
    pdf.cell(200, 10, txt=header, ln=True, align='C')
    pdf.ln(1)  # Set line break to 1

    for company in list_name:
        if company.isdigit():
            company_int = int(company)
            company_data = combined_df[combined_df['BSE_Code'] == company_int]
        else:
            company = company.upper()
            company_data = combined_df[combined_df['NSE_Code'] == company]

        if not company_data.empty:
            company_name = company_data['Company Name'].values[0]
            nse_code = company_data['NSE_Code'].values[0]
            bse_code = company_data['BSE_Code'].values[0]

            ema10, ema20, ema50, ema200 = averages(nse_code)
            ema_str = "ema10 / ema20 / ema50 / ema200"

            mktcap_value = get_mktcap(nse_code)

            # Add company information to PDF
            pdf.cell(0, 8, txt=f"Company Name: {company_name} (NSE Code: {nse_code}, BSE Code: {bse_code}, Mktcap: {mktcap_value[0]:,.0f})", ln=True)
            pdf.cell(0, 8, txt=f"{ema_str}     {ema10[0]}  {ema20[0]}  {ema50[0]}  {ema200[0]}", ln=True)
            pdf.ln(2)  # Set line break to 2

            formatted_columns = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
            formatted_data = company_data[formatted_columns].copy()
            formatted_data['NSE_Delivery'] = formatted_data['NSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['BSE_Delivery'] = formatted_data['BSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Total_Delivery'] = formatted_data['Total_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{x:.2f}")

            formatted_data_str = formatted_data.to_string(index=False, col_space={'Date': 9, 'NSE_Delivery': 15, 'BSE_Delivery': 15,
                                                                                   'Total_Delivery': 15, 'Price': 10})

            pdf.multi_cell(0, 8, txt=formatted_data_str)
            pdf.cell(0, 8, txt="\n" + "=" * 50 + "\n", ln=True)  # Set line break to 1
        else:
            pdf.cell(0, 8, txt=f"No data found for {company}\n", ln=True)

    # Output the PDF to a file
    pdf.output("output_a4_spacing_1_reduced.pdf")

# Example usage:
list_process_to_pdf(check_stock, "temp check")
